In [1]:
!nvidia-smi

Thu Jul  8 11:23:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/EMNLP_folder_4/dataset/

/content/drive/.shortcut-targets-by-id/1ANtRsCBtHNhrdxlObGcNtnE59DOMZyhS/EMNLP_folder_4/dataset


In [4]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q
!pip install neptune-client -q

     |████████████████████████████████| 2.5MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 49.4MB/s 
     |████████████████████████████████| 901kB 52.1MB/s 
     |████████████████████████████████| 1.2MB 9.3MB/s 
     |████████████████████████████████| 245kB 7.6MB/s 
     |████████████████████████████████| 829kB 13.6MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 174kB 21.0MB/s 
     |████████████████████████████████| 133kB 26.6MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 


# Importing the Required Libraries

In [5]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

from pandarallel import pandarallel
# Initialization
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
run = neptune.init(project='manav0211/emnlp',
                   tags = 'roberta large seed 2 training on sts before the main dataset',
                   api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMzUxMGNhMS00N2E5LTQ2YmUtYWI1Yi03ZDFhZTEzODg1NmEifQ==') # your credentials

https://app.neptune.ai/manav0211/emnlp/e/EM-354
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [7]:
SEED = 2
rn.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = 'cuda'

In [8]:
path_dataset = '/content/drive/MyDrive/EMNLP_folder_4/dataset/'

# use the below two lines for storing model that is trained on STS dataset first and comment the next two lined
path_predictions_folder = '/content/drive/MyDrive/EMNLP_folder_4/trained_on_sts_then_on_combined_dataset_model_predictions/'
save_model_folder = '/content/drive/MyDrive/EMNLP_folder_4/trained_on_sts_then_on_combined_dataset_model_files/'

# use the below two lines for storing model that is trained on Args30k dataset first and comment the previous two lines

#path_predictions_folder = '/content/drive/MyDrive/EMNLP_folder_4/trained_on_arg30k_then_on_combined_dataset_model_predictions/'
#save_model_folder = '/content/drive/MyDrive/EMNLP_folder_4/trained_on_arg30k_then_on_combined_dataset_model_files/'


In [9]:
max_len_arg = 55
max_len_kp = 32
max_len_topic = 12
max_len_sent_1_sts = 58
max_len_sent_2_sts = 65
median_len_sent_1_sts = 9
median_len_sent_2_sts = 9
max_noun_encoded_feature_len = 66
max_dependency_encoded_feature_len = 66
max_len_tf_idf = 505
max_len_input = 128
model_with_no_token_types =['roberta', 'bart', 'albert', 'distilbert','deberta', 'xlmroberta', 'xlnet','xlnetlarge', 'robertalarge', 'bartlarge', 'debertalarge', 'xlmrobertalarge', 'albertlarge']

### Function to make TensorDataset of the files

In [10]:
def make_dataset(tokenizer, args,kps,topics, labels, max_len_input, model_with_no_token_types = model_with_no_token_types, model_name='roberta'):
    
    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = [] 
    
    for arg,kp,topic,label in zip(args,kps,topics,labels) :

        arg = re.sub('[^a-zA-Z]', ' ', arg)
        kp = re.sub('[^a-zA-Z]', ' ', kp)
        topic = re.sub('[^a-zA-Z]', ' ', topic)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'',arg)
        kp = url.sub(r'',kp)
        topic = url.sub(r'',topic)
        
        html=re.compile(r'<.*?>')
        arg = html.sub(r'',arg)
        kp = html.sub(r'',kp)
        topic = html.sub(r'',topic)


        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        
        arg = emoji_pattern.sub(r'',arg)
        kp = emoji_pattern.sub(r'',kp)
        topic = emoji_pattern.sub(r'',topic)

        if model_name in model_with_no_token_types:

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)

        else :

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
          
    if model_name in model_with_no_token_types:
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels)
      
      dataset = TensorDataset(all_input_ids, all_attention_masks, all_labels)

    else :
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels) 

      dataset = TensorDataset(all_input_ids,all_token_type_ids, all_attention_masks, all_labels)

    return dataset

#### Use the below function to make dataset for Args30k and STS dataset

In [11]:
def make_dataset_additional(tokenizer, sents_1,sents_2, labels, max_len_input, model_with_no_token_types = model_with_no_token_types, model_name='roberta'):
    
    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = [] 
    
    for arg,kp,label in zip(sents_1,sents_2, labels) :

        arg = re.sub('[^a-zA-Z]', ' ', arg)
        kp = re.sub('[^a-zA-Z]', ' ', kp)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'',arg)
        kp = url.sub(r'',kp)
        
        html=re.compile(r'<.*?>')
        arg = html.sub(r'',arg)
        kp = html.sub(r'',kp)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        
        arg = emoji_pattern.sub(r'',arg)
        kp = emoji_pattern.sub(r'',kp)

        if model_name in model_with_no_token_types:

          encoded_input = tokenizer(kp+arg,max_length = max_len_input, padding='max_length',truncation='longest_first')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)

        else :

          encoded_input = tokenizer(kp+arg,max_length = max_len_input, padding='max_length',truncation='longest_first')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
          
    if model_name in model_with_no_token_types:
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels)
      
      dataset = TensorDataset(all_input_ids, all_attention_masks, all_labels)

    else :
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels) 

      dataset = TensorDataset(all_input_ids,all_token_type_ids, all_attention_masks, all_labels)

    return dataset

## Loading the Dataset

In [12]:
df_sts = pd.read_csv(path_dataset+'sts_dataset.csv')
df_arg30 = pd.read_csv(path_dataset+'30k_dataset.csv')
df_train = pd.read_csv(path_dataset+'train_tfidf.csv')
df_val = pd.read_csv(path_dataset+'val_tfidf.csv')
df_test  = pd.read_csv(path_dataset+'final_test.csv')

print(df_train.shape,df_val.shape,df_test.shape,df_sts.shape,df_arg30.shape)

df_train.head()

(20635, 31) (3458, 31) (3923, 18) (8020, 16) (30497, 8)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,arg_id,key_point_id,label,arg,key_point,topic,stance,t5_key_point,arg_token_len,kp_token_len,t5_kp_token_len,topic_token_len,kp_arg_topic,dependency_features,noun_features,encoded_dependency_features,encoded_noun_features,encoded_noun_features_len,encoded_dependency_features_len,tf_idf_features,tf_idf_len,tf_idf_bigram_features,tf_idf_bigram_len,tf_idf_trigram_features,tf_idf_trigram_len
0,0,0,0,0,0,0,arg_0_0,kp_0_0,0,`people reach their limit when it comes to the...,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1,a person who wants to commit it. assisted suic...,47,13,37,10,Assisted suicide gives dignity to the person t...,"['amod', 'nsubj', 'ROOT', 'dobj', 'dative', 'd...","['NOUN', 'VERB', 'VERB', 'VERB', 'VERB', 'NOUN...","[1, 2, 3, 4, 5, 6, 7, 2, 8, 9, 10, 4, 11, 12, ...","[1, 2, 2, 2, 2, 1, 3, 2, 1, 2, 2, 2, 2, 1, 2, ...",60,60,"[0.5073060076370642, 0.0, 0.0, 0.0, 0.0, 0.0, ...",298,"[0.419428095234635, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",329,"[0.38897968136796685, 0.0, 0.0, 0.0, 0.0, 0.0,...",684
1,1,1,1,1,1,1,arg_0_121,kp_0_4,0,a cure or treatment may be discovered shortly ...,Assisted suicide allows people to solicit some...,Assisted suicide should be a criminal offence,1,allows people to seek someone out to die to th...,24,14,43,10,Assisted suicide allows people to solicit some...,"['amod', 'nsubj', 'ROOT', 'nsubj', 'aux', 'cco...","['NOUN', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB...","[1, 2, 3, 2, 9, 22, 4, 9, 8, 15, 12, 1, 23, 7,...","[1, 2, 2, 2, 2, 2, 2, 2, 7, 2, 1, 1, 1, 3, 1, ...",35,35,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",298,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",329,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",684
2,2,2,2,2,2,2,arg_0_121,kp_0_5,0,a cure or treatment may be discovered shortly ...,Assisted suicide is akin to killing someone,Assisted suicide should be a criminal offence,1,is similar to assisted suicide. assisted suici...,24,10,39,10,Assisted suicide is akin to killing someonea c...,"['amod', 'nsubj', 'ROOT', 'acomp', 'prep', 'pc...","['NOUN', 'AUX', 'AUX', 'AUX', 'ADJ', 'ADP', 'N...","[1, 2, 3, 17, 15, 24, 23, 4, 16, 11, 9, 20, 11...","[1, 4, 4, 4, 5, 3, 1, 2, 1, 1, 2, 2, 4, 3, 2, ...",29,29,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31489661...",298,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",329,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",684
3,3,3,3,3,3,3,arg_0_121,kp_0_6,1,a cure or treatment may be discovered shortly ...,Assisted suicide should not be allowed because...,Assisted suicide should be a criminal offence,1,because many people can still get better. assi...,24,15,45,10,Assisted suicide should not be allowed because...,"['amod', 'nsubjpass', 'aux', 'neg', 'auxpass',...","['NOUN', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB...","[1, 19, 9, 26, 20, 3, 27, 1, 28, 2, 9, 13, 14,...","[1, 2, 2, 2, 2, 2, 4, 1, 4, 4, 4, 4, 2, 1, 4, ...",36,36,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",298,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",329,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",684
4,4,4,4,4,4,4,arg_0_121,kp_0_7,0,a cure or treatment may be discovered shortly ...,Assisted suicide violates the doctor's role,Assisted suicide should be a criminal offence,1,suicide is an assisted suicide. is an assisted...,24,10,39,10,Assisted suicide violates the doctor's rolea c...,"['amod', 'nsubj', 'ROOT', 'det', 'poss', 'case...","['NOUN', 'VERB', 'VERB', 'NOUN', 'NOUN', 'NOUN...","[1, 2, 3, 6, 12, 25, 23, 19, 16, 11, 9, 20, 22...","[1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 3, 2, ...",29,29,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",298,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",329,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",684


In [13]:
df_sts.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,sent_1,sent_2,label,label_normalized,sent_1_token_lengths,sent_2_token_lengths,input_sentence,dependency_features,noun_features,encoded_dependency_features,encoded_noun_features,encoded_noun_features_len,encoded_dependency_features_len
0,0,0,0,A plane is taking off.,An air plane is taking off.,5.00,1.00,6,7,A plane is taking off.An air plane is taking off.,"['det', 'nsubj', 'aux', 'ROOT', 'prt', 'punct'...","['NOUN', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB...","[1, 2, 3, 4, 5, 6, 1, 7, 2, 3, 4, 5, 6, 0, 0, ...","[1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 0, ...",84,84
1,1,1,1,A man is playing a large flute.,A man is playing a flute.,3.80,0.76,9,8,A man is playing a large flute.A man is playin...,"['det', 'nsubj', 'aux', 'ROOT', 'det', 'amod',...","['NOUN', 'VERB', 'VERB', 'VERB', 'NOUN', 'NOUN...","[1, 2, 3, 4, 1, 8, 9, 6, 1, 2, 3, 4, 1, 9, 6, ...","[1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, ...",84,84
2,2,2,2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80,0.76,11,12,A man is spreading shreded cheese on a pizza.A...,"['det', 'nsubj', 'aux', 'ROOT', 'amod', 'dobj'...","['NOUN', 'VERB', 'VERB', 'VERB', 'NOUN', 'VERB...","[1, 2, 3, 4, 8, 9, 10, 1, 11, 6, 1, 2, 3, 4, 8...","[1, 2, 2, 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 1, ...",84,84
3,3,3,3,Three men are playing chess.,Two men are playing chess.,2.60,0.52,6,6,Three men are playing chess.Two men are playin...,"['nummod', 'nsubj', 'aux', 'ROOT', 'dobj', 'pu...","['NOUN', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB...","[12, 2, 3, 4, 9, 6, 12, 2, 3, 4, 9, 6, 0, 0, 0...","[1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, ...",84,84
4,4,4,4,A man is playing the cello.,A man seated is playing the cello.,4.25,0.85,8,9,A man is playing the cello.A man seated is pla...,"['det', 'nsubj', 'aux', 'ROOT', 'det', 'dobj',...","['NOUN', 'VERB', 'VERB', 'VERB', 'NOUN', 'VERB...","[1, 2, 3, 4, 1, 9, 6, 1, 2, 13, 3, 4, 1, 9, 6,...","[1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, ...",84,84


In [14]:
df_arg30.head()

,Unnamed: 0,arg,topic,set,WA,label,stance_WA,stance_WA_conf
0,0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,train,0.846165,0.297659,1,1.000000
1,1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,train,0.891271,0.726133,-1,1.000000
2,2,\ero-tolerance policy in schools should not be...,We should adopt a zero-tolerance policy in sch...,dev,0.721192,0.396953,-1,1.000000
3,3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,train,0.730395,0.225212,-1,1.000000
4,4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,train,0.236686,0.004104,1,0.805517


In [15]:
# Here I am concatenating train and dev set for training the model for creating test dataset predictions
df_train = pd.concat([df_train,df_val])
df_train.shape

(24093, 31)

types_of_models= model : tokenizer model_path
 - 'bert':  'bert-base-uncased'
 - 'roberta':  'roberta-base'
 - 'bart':  "facebook/bart-base"
 - 'distilbert': 'distilbert-base-uncased'
 - 'deberta': 'microsoft/deberta-base'
 - 'debertalarge': 'microsoft/deberta-large'
 - 'xlnet' : 'xlnet-base-cased'
 - 'xlnetlarge' : 'xlnet-large-cased'
 - 'xlmrobertalarge' : 'xlm-roberta-large'
 - 'bartlarge' : 'facebook/bart-large'
 - 'bertlarge':  'bert-large-uncased'
 - 'robertalarge':  'roberta-large'

In [16]:
model_name = 'robertalarge'
model_path = 'roberta-large'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [18]:
## uncomment the first line to make the tensor dataset for STS dataset and the second line for the Args30k dataset

sts_train_dataset = make_dataset_additional(tokenizer, df_sts['sent_1'],df_sts['sent_2'], df_sts['label_normalized'], max_len_input,model_with_no_token_types, model_name=model_name)
#args30k_train_dataset = make_dataset_additional(tokenizer, df_arg30['arg'],df_arg30['topic'], df_arg30['label'], max_len_input,model_with_no_token_types, model_name=model_name)

train_dataset = make_dataset(tokenizer, df_train['arg'], df_train['key_point'], df_train['topic'], df_train['label'], max_len_input, model_with_no_token_types, model_name=model_name)
val_dataset = make_dataset(tokenizer, df_val['arg'], df_val['key_point'], df_val['topic'], df_val['label'], max_len_input, model_with_no_token_types, model_name=model_name)
test_dataset = make_dataset(tokenizer, df_test['arg'], df_test['key_point'], df_test['topic'], df_test['stance'], max_len_input, model_with_no_token_types, model_name=model_name)

# Model Architecture

In [19]:
# Use this Class only for Bert base and Bert large model

class Transformer(nn.Module):

    def __init__(self):
        super(Transformer, self).__init__()
        
        #Instantiating Pre trained model object 
        self.model_layer = AutoModel.from_pretrained(model_path)
        
        #Layers
        # the first dense layer will have 768 neurons if base model is used and 
        # 1024 neurons if large model is used

        self.dense_layer_1 = nn.Linear(1024, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.4) 
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks, token_type_ids):

        pooled_output = self.model_layer(input_ids=input_ids, attention_mask=attention_masks,token_type_ids = token_type_ids).pooler_output
        
        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)
        
        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [20]:
# Use this Class for the rest of transformer models

class NonPoolerTransformer(nn.Module):

    def __init__(self):
        super(NonPoolerTransformer, self).__init__()
        
        #Instantiating Pre trained model object 
        self.model_layer = AutoModel.from_pretrained(model_path)

        #Layers
        # the first dense layer will have 768 if base model is used and 
        # 1024 if large model is used

        self.dense_layer_1 = nn.Linear(1024, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.2)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks):

        hidden_state = self.model_layer(input_ids=input_ids, attention_mask=attention_masks)[0]
        pooled_output = hidden_state[:, 0]

        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [21]:
# comment the nonpoolertransformer line for bert model and the transformer model line for the rest of the transformer models
model = NonPoolerTransformer()
#model = Transformer()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
model = model.to(device)

In [23]:
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
EPOCHS = 3
ACCUMULATION_STEPS = 2
DROPOUT = 0.4
gold_data_dir = path_dataset

In [24]:
PARAMS = {'model_name': model_name,'model_path': model_path,'lr': LEARNING_RATE, 'epoch_nr': EPOCHS, 'batch_size': BATCH_SIZE, 'accumulation_steps': ACCUMULATION_STEPS,'dropout': DROPOUT}
run['details'] = PARAMS

### Evaluation functions

In [25]:
def load_kpm_data(gold_data_dir, subset):
    
    arguments_file = os.path.join(gold_data_dir, f"arguments_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_{subset}.csv")
    labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    labels_file_df = pd.read_csv(labels_file)
    
    return arguments_df, key_points_df, labels_file_df

In [26]:
def get_predictions(predictions_file, labels_df, arg_df, kp_df):
    #print("\nֿ** loading predictions:")
    arg_df = arg_df[["arg_id", "topic", "stance"]]
    predictions_df = load_predictions(predictions_file, kp_df["key_point_id"].unique())

    #make sure each arg_id has a prediction
    predictions_df = pd.merge(arg_df, predictions_df, how="left", on="arg_id")

    #handle arguements with no matching key point
    predictions_df["key_point_id"] = predictions_df["key_point_id"].fillna("dummy_id")
    predictions_df["score"] = predictions_df["score"].fillna(0)

    #merge each argument with the gold labels
    merged_df = pd.merge(predictions_df, labels_df, how="left", on=["arg_id", "key_point_id"])

    merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
    merged_df["label_strict"] = merged_df["label"].fillna(0)
    merged_df["label_relaxed"] = merged_df["label"].fillna(1)

    return merged_df

In [27]:
def load_predictions(predictions_dir, correct_kp_list):
    arg =[]
    kp = []
    scores = []
    invalid_keypoints = set()
    with open(predictions_dir, "r") as f_in:
        res = json.load(f_in)
        for arg_id, kps in res.items():
            valid_kps = {key: value for key, value in kps.items() if key in correct_kp_list}
            invalid = {key: value for key, value in kps.items() if key not in correct_kp_list}
            for invalid_kp, _ in invalid.items():
                if invalid_kp not in invalid_keypoints:
                    #print(f"key point {invalid_kp} doesn't appear in the key points file and will be ignored")
                    invalid_keypoints.add(invalid_kp)
            if valid_kps:
                best_kp = max(valid_kps.items(), key=lambda x: x[1])
                arg.append(arg_id)
                kp.append(best_kp[0])
                scores.append(best_kp[1])
        #print(f"\tloaded predictions for {len(arg)} arguments")
        
        return pd.DataFrame({"arg_id" : arg, "key_point_id": kp, "score": scores})

In [28]:
def get_ap(df, label_column, top_percentile=0.5):
    top = int(len(df)*top_percentile)
    df = df.sort_values('score', ascending=False).head(top)
    # after selecting top percentile candidates, we set the score for the dummy kp to 1, to prevent it from increasing the precision.
    df.loc[df['key_point_id'] == "dummy_id", 'score'] = 0.99
    return average_precision_score(y_true=df[label_column], y_score=df["score"])

def calc_mean_average_precision(df, label_column):
    precisions = [get_ap(group, label_column) for _, group in df.groupby(["topic", "stance"])]
    return np.mean(precisions)

def evaluate_predictions(merged_df,name = 'train'):
    #print("\n** running evalution:")
    mAP_strict = calc_mean_average_precision(merged_df, "label_strict")
    mAP_relaxed = calc_mean_average_precision(merged_df, "label_relaxed")
    # below two lines are added for neptune results logging
    run[f"{name}/map"].log(mAP_strict)
    run[f"{name}/map_relaxed"].log(mAP_relaxed)
                         
    print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

### Train and Predict Functions

In [29]:
def evaluate_model(test_dataset,df, model,  model_name, mode = 'train'):
    
    save_predictions_name = model_name+ '__VAL_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(EPOCHS)+'_lr_'+ str(LEARNING_RATE)+'_b_s_'+ str(BATCH_SIZE) +'_accumulation_steps_'+ str(ACCUMULATION_STEPS) +'_input_type_kp_arg_topic_'

    y_preds = []
    val_losses = []
    criterion = nn.BCELoss()
    list_of_batch_losses = []
    
    if mode in ['train','val']:
        test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    with torch.no_grad():
        acc_epoch = []
        
        epoch_iterator = tqdm(test_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.eval()
            
            if model_name in model_with_no_token_types:
                b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
                ypred = model(b_input_ids, b_input_mask)
            else:
                b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
                ypred = model(b_input_ids, b_input_mask,b_token_type)
                
            b_labels_copy = torch.reshape(b_labels, (b_labels.shape[0], 1))
            loss_batch = criterion(ypred, b_labels_copy.float())
            list_of_batch_losses.append(loss_batch.detach().cpu().numpy())
            run["val/batch_loss"].log(np.mean(loss_batch.detach().cpu().numpy()))
            
            ypred = ypred.cpu().numpy()
            b_labels = batch[-1].cpu().detach().numpy()
        
            ypred = np.hstack(ypred)
            y_preds.append(ypred)
    
    epoch_loss = np.mean(list_of_batch_losses)
    val_losses.append(epoch_loss)
    run["val/epoch_loss"].log(epoch_loss)
    
    args = df['arg_id']
    kps = df['key_point_id']
    true_labels = df['label']
    topics = df['topic']
    stances = df['stance']
    all_preds = []

    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)
            
    print('Val evaluation....')
    
    pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})
    args = {}
    kps = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg] = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg][kp] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()
    
    arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="dev")
    merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)
    
    evaluate_predictions(merged_df,name = 'val')

    return all_preds,true_labels, val_losses

In [30]:
def train_and_evaluate(train_dataset,df,model, filepath, model_name, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS,accumulation_steps = ACCUMULATION_STEPS):
  
  train_losses = []
  val_losses = []
    
  save_model = model_name+ '_SEED_'+ str(SEED) +'_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_kp_arg_topic' 
  save_predictions_name  = model_name+ '__TRAIN_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_kp_arg_topic'

  training_dataloader = DataLoader(train_dataset, batch_size )
  total_steps = len(training_dataloader) * epochs
  no_decay = ['bias', 'LayerNorm.weight']
  
  optimizer_grouped_parameters = [
                                  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                                  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

  criterion = nn.BCELoss()
    
  model.zero_grad()

  for epoch_i in tqdm(range(epochs)):
    y_preds = []
    y_val = []
    list_of_batch_losses = []
    epoch_iterator = tqdm(training_dataloader, desc="Iteration")
    model.train()
    
    for step, batch in enumerate(epoch_iterator):
      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(b_input_ids, b_input_mask)
      else:
        b_input_ids,b_token_type, b_input_mask,b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(b_input_ids, b_input_mask,b_token_type)
            
      b_labels = torch.reshape(b_labels, (b_labels.shape[0], 1))
      loss = criterion(outputs, b_labels.float())
             
      list_of_batch_losses.append(loss.detach().cpu().numpy())
      run["train/batch_loss"].log(np.mean(loss.detach().cpu().numpy()))
      
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
      ypred = outputs.detach().cpu().numpy()
      b_labels = batch[-1].cpu().detach().numpy()
      ypred = np.hstack(ypred)
      y_preds.append(ypred)

      if (step+1) % accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        model.zero_grad()


    epoch_loss = np.mean(list_of_batch_losses)
    train_losses.append(epoch_loss)
    run["train/epoch_loss"].log(epoch_loss)
    
    args = df['arg_id']
    kps = df['key_point_id']
    true_labels = df['label']
    topics = df['topic']
    stances = df['stance']
    all_preds = []
    
    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)

    print('Train evaluation....')
    
    pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})
    args = {}
    kps = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg] = {}

    for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
        args[arg][kp] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()
    
    arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="train")
    merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)
    
    evaluate_predictions(merged_df,name = 'train')
    
    _,_, val_epoch_loss = evaluate_model(val_dataset,df_val, model,  model_name, mode = 'val')
    val_losses.append(val_epoch_loss)
    

  torch.save(model, save_model_folder +save_model+'.pt')
  run["model"].upload(save_model_folder +save_model+'.pt')

  print("Model is saved as : ",save_model)
  print("Use this to load the model")
    
  return save_model,train_losses, val_losses

In [31]:
def predict_model(test_dataset,df, save_model,model_name):
  preds = []

  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
  save_predictions_name = "TEST_PREDS_"+ '_SEED_'+ str(SEED) + save_model

  model=torch.load(save_model_folder + save_model +'.pt')

  with torch.no_grad():
    
    epoch_iterator = tqdm(test_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.eval()

      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        ypred = model(b_input_ids, b_input_mask)
      else:
        b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        ypred = model(b_input_ids, b_input_mask,b_token_type)

      ypred = ypred.cpu().numpy()
      ypred = np.hstack(ypred)

      preds.append(ypred)

  args = df['arg_id']
  kps = df['key_point_id']
  all_preds = []

  for i in tqdm(range(len(preds))):
    for p in preds[i]:
      all_preds.append(p)

  pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})

  args = {}
  kps = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg] = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg][kp] = score

  with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
    fp.write(json.dumps(args))
    fp.close()

  print("The predictions are stored in the file : "+ path_predictions_folder  + save_predictions_name + '_' + 'predictions.p.')
  
  return path_predictions_folder + save_predictions_name + '_' + 'predictions.p.'

In [32]:
def give_test_results(pred_file_path):
  print('The strict and relaxed scores on the test set predictions are: ')
  arg_df, kp_df, labels_df = load_kpm_data(gold_data_dir, subset="test")
  merged_df = get_predictions(pred_file_path, labels_df, arg_df, kp_df)
  evaluate_predictions(merged_df)

#### Use the function below to train additional dataset such as STS or Args30k

In [33]:
def train_additional_dataset(train_dataset, model, filepath, model_name, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS,accumulation_steps = 1):
  
  losses = []
  save_model = model_name+ '_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) + '_input_type_one'

  training_dataloader = DataLoader(train_dataset, batch_size )
  total_steps = len(training_dataloader) * epochs
  no_decay = ['bias', 'LayerNorm.weight']
  
  optimizer_grouped_parameters = [
                                  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                                  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
  criterion = nn.BCELoss()
    
  model.zero_grad()
  for epoch_i in tqdm(range(epochs)):

    epoch_iterator = tqdm(training_dataloader, desc="Iteration")
    model.train()

    for step, batch in enumerate(epoch_iterator):
      
      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(b_input_ids, b_input_mask)
      
      else:
        b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(b_input_ids, b_input_mask,b_token_type)
          
   
      b_labels = torch.reshape(b_labels, (b_labels.shape[0], 1))

      loss = criterion(outputs, b_labels.float())
      loss = loss / accumulation_steps  
      losses.append(loss)
      
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      if (step+1) % accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        model.zero_grad()
 
  #torch.save(model, save_model_folder +save_model+'.pt')
  #print("Model is saved as : ",save_model)
  print("Model training on additional dataset is finished, but model not stored, since required ahead")

  return save_model,losses

In [34]:
## train on sts for 5 epochs and on args30k for 3 epochs
save_model, train_additional_dataset_losses = train_additional_dataset(sts_train_dataset, model, save_model_folder, model_name = 'robertalarge', batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = 5, accumulation_steps = ACCUMULATION_STEPS)



Model training on additional dataset is finished, but model not stored, since required ahead


In [35]:
save_model, train_losses, val_losses = train_and_evaluate(train_dataset,df_train, model, save_model_folder, model_name = 'robertalarge', batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS, accumulation_steps = ACCUMULATION_STEPS)


Train evaluation....
mAP strict= 0.8721083524365504 ; mAP relaxed = 0.8721083524365504



Val evaluation....
mAP strict= 0.9414744089244601 ; mAP relaxed = 0.9414744089244601



Train evaluation....
mAP strict= 0.9201987079539262 ; mAP relaxed = 0.9201987079539262



Val evaluation....
mAP strict= 0.991447064657947 ; mAP relaxed = 0.991447064657947



Train evaluation....
mAP strict= 0.9601729220001157 ; mAP relaxed = 0.9601729220001157



Val evaluation....
mAP strict= 0.9966832098845101 ; mAP relaxed = 0.9966832098845101

Model is saved as :  robertalarge_SEED_2_dense_layer_epoc_3_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic
Use this to load the model


In [36]:
test_preds_path = predict_model (test_dataset,df_test, save_model,model_name = 'robertalarge')


The predictions are stored in the file : /content/drive/MyDrive/EMNLP_folder_4/trained_on_sts_then_on_combined_dataset_model_predictions/TEST_PREDS__SEED_2robertalarge_SEED_2_dense_layer_epoc_3_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic_predictions.p.


In [37]:
give_test_results(test_preds_path)

The strict and relaxed scores on the test set predictions are: 
mAP strict= 0.9055571033203983 ; mAP relaxed = 0.9828347328900433
